In [1]:
# add additional module import path to import the desired function from the classifier module
# https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im#answer-35273613
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
# load classifier dependencies
from classifier.lib import Data
from classifier.linguistic.lookupdict import LookUpDict

# define data path
DATA_PATH: str = '../data'

In [7]:
# load train data, set polarities, and config (ngrams, stopwords, lemmas)
train: Data = Data(
    f'{DATA_PATH}/imdb.train.1.000.csv',
    polarities={
      "negative": 0,
      "positive": 1
    },
    config={
      "generate_ngrams": [
        1,
        2
      ],
      "remove_stopwords": True,
      "use_lemmatizer": False
    }
)
train.data.head(16)

,review,sentiment,token,2-gram
0,Dave is going through a divorce and his mind w...,negative,"[dave, going, divorce, mind, wanders, back, fi...","[(dave, going), (going, divorce), (divorce, mi..."
1,"Highly regarded at release, but since rather n...",positive,"[highly, regarded, release, since, rather, neg...","[(highly, regarded), (regarded, release), (rel..."
2,this western/musical/comedy is not one of the ...,negative,"[western, musical, comedy, one, best, genre, s...","[(western, musical), (musical, comedy), (comed..."
3,I opted to watch this film for one reason and ...,positive,"[opted, watch, film, one, reason, one, reason,...","[(opted, watch), (watch, film), (film, one), (..."
4,"As one other IMDb reviewer puts it, ""...imagin...",positive,"[one, imdb, reviewer, puts, imagine, space, od...","[(one, imdb), (imdb, reviewer), (reviewer, put..."
5,boring stuff we got here. His 5 minute shorts ...,negative,"[boring, stuff, got, minute, shorts, better, k...","[(boring, stuff), (stuff, got), (got, minute),..."
6,"This is a well directed Columbo episode, with ...",positive,"[well, directed, columbo, episode, also, good,...","[(well, directed), (directed, columbo), (colum..."
7,I was watching this movie and getting increasi...,negative,"[watching, movie, getting, increasingly, bored...","[(watching, movie), (movie, getting), (getting..."
8,Sporting a title seemingly more suitable for a...,positive,"[sporting, title, seemingly, suitable, looney,...","[(sporting, title), (title, seemingly), (seemi..."
9,Curl up with this one on a dark and stormy nig...,positive,"[curl, one, dark, stormy, night, prepare, alte...","[(curl, one), (one, dark), (dark, stormy), (st..."


In [4]:
lookUpUni = LookUpDict({
    'data': train.data,
    'key_label': 'token',
    'group_label': 'sentiment',
    'pre_selection': 1024,
    'final_selection': 64
})
lookUpUni


name(negative) || len(64) || sum(57866) 
            token     n         p
52         worst  3956  0.068365
110        awful  2474  0.042754
125     terrible  2349  0.040594
151        waste  2062  0.035634
158        worse  2000  0.034563
195     horrible  1706  0.029482
244         crap  1431  0.024730
293     annoying  1238  0.021394
300   ridiculous  1215  0.020997
327        cheap  1110  0.019182
338         dull  1077  0.018612
342        clich  1067  0.018439
365        avoid  1019  0.017610
370  predictable  1012  0.017489
374       poorly  1000  0.017281
384         lame   977  0.016884
name(positive) || len(64) || sum(39981) 
            token    n         p
368    fantastic  987  0.024687
419     greatest  894  0.022361
420       superb  893  0.022336
434    realistic  860  0.021510
436        oscar  859  0.021485
445         tale  841  0.021035
488       season  779  0.019484
494  masterpiece  774  0.019359
502       unique  766  0.019159
509     powerful  761  0.019034
51

In [5]:
lookUpBi = LookUpDict({
    'data': train.data,
    'key_label': "2-gram",
    'group_label': 'sentiment',
    'pre_selection': 1024,
    'final_selection': 64
})
lookUpBi


name(negative) || len(64) || sum(14136) 
               2-gram     n         p
2      (waste, time)  1129  0.079867
6       (one, worst)   788  0.055744
16    (worst, movie)   570  0.040323
19     (bad, acting)   556  0.039332
43     (even, worse)   378  0.026740
55       (bad, film)   342  0.024194
60   (worst, movies)   335  0.023698
67     (acting, bad)   316  0.022354
77     (worst, film)   305  0.021576
87     (bad, movies)   282  0.019949
108         (mst, k)   255  0.018039
109       (bad, bad)   255  0.018039
111    (time, money)   252  0.017827
123    (plot, holes)   237  0.016766
133      (film, bad)   230  0.016271
136     (stay, away)   228  0.016129
name(positive) || len(64) || sum(10896) 
                     2-gram    n         p
27     (highly, recommend)  403  0.036986
34           (well, worth)  367  0.033682
50            (great, job)  319  0.029277
62           (love, movie)  300  0.027533
61         (well, written)  300  0.027533
66   (highly, recommended)  295  0